# Data Preprocessing for training data set
Here we check the training URLs for any duplicates and reduce the dimesnsions of the training data so as to keep only the important features.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd

In [3]:
#loading dataset
trainingdata = pd.read_csv("/content/drive/MyDrive/URL_classification using ML/Training/Data/trainingdata(short).csv",encoding = "ISO-8859-1")

In [4]:
trainingdata.head()

,domain,ranking,mld_res,mld.ps_res,card_rem,ratio_Rrem,ratio_Arem,jaccard_RR,jaccard_RA,jaccard_AR,jaccard_AA,jaccard_ARrd,jaccard_ARrem,label
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,10000000,1,0,18,107.611111,107.277778,0.0,0.0,0.0,0.0,0.8,0.795729,1
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,10000000,0,0,11,150.636364,152.272727,0.0,0.0,0.0,0.0,0.0,0.768577,1
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,10000000,0,0,14,73.500000,72.642857,0.0,0.0,0.0,0.0,0.0,0.726582,1
3,mail.printakid.com/www.online.americanexpress....,10000000,0,0,6,562.000000,590.666667,0.0,0.0,0.0,0.0,0.0,0.859640,1
4,thewhiskeydregs.com/wp-content/themes/widescre...,10000000,0,0,8,29.000000,24.125000,0.0,0.0,0.0,0.0,0.0,0.748971,1


In [5]:
trainingdata.shape

(501, 14)

In [6]:
#taking only required columns into a new Dataframe
trainingdata = trainingdata.filter(['domain','label'],axis=1)

In [7]:
trainingdata.head()

,domain,label
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,1
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,1
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,1
3,mail.printakid.com/www.online.americanexpress....,1
4,thewhiskeydregs.com/wp-content/themes/widescre...,1


In [8]:
#Checking for NULL values
trainingdata.duplicated().sum()

0

In [9]:
#removing duplicates
trainingdata = trainingdata.drop_duplicates()

In [10]:
trainingdata.shape

(501, 2)

In [11]:
trainingdata.to_csv("/content/drive/MyDrive/URL_classification using ML/Training/Data/preprocessed_trainingdata(short).csv",index=False)

# Whois Verification of training URLs
Here we use python.whois tool to check how many URLs out of the preprocessed URLs are reachable by whois and merge those in a csv file.



## 1. Verifying Benign URLs

In [12]:
!pip install python-whois

     |████████████████████████████████| 91 kB 4.5 MB/s 
  Created wheel for python-whois: filename=python_whois-0.7.3-py3-none-any.whl size=87720 sha256=8b2dc4b927ebd03bce27ac98de4bf3c74b53138257ee15816de672c99fc18a34
  Stored in directory: /root/.cache/pip/wheels/11/05/f7/895ce5a73665f77c8274a7d55e34fb3e6b4abbb9a7637e215b
Successfully built python-whois


In [13]:
import numpy as np
import pandas as pd
import whois

In [14]:
trainingdata = pd.read_csv("/content/drive/MyDrive/URL_classification using ML/Training/Data/preprocessed_trainingdata(short).csv",encoding = "ISO-8859-1")

In [15]:
#this function performs whois and returns true to those urls which are reachable through whois
def performwhois(url):
    try:
        result = whois.whois(url)
        return result #success
    except Exception:
        return False #error

In [16]:
benign_sample = trainingdata['domain'][252:503]

In [17]:
#store the benign URLs in a numpy array which are reachable from whois
benign_urls = []
counter = 0
for url in benign_sample:
    if performwhois(url):
        counter = counter + 1
        print(counter)
        benign_urls.append(url)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248


In [18]:
benign_df = pd.DataFrame()

In [19]:
benign_df['url'] = benign_urls

In [20]:
benign_df['label'] = 0

In [21]:
benign_df.to_csv("/content/drive/MyDrive/URL_classification using ML/Training/Data/whois_verified_benign_training(short).csv",index=False)

## 2. Verifying Phishing URLs

In [ ]:
phishing_sample = trainingdata['domain'][:250]

In [ ]:
#store the phishing URLs in a numpy array which are reachable from whois
phishing_urls = []
counter = 0
for url in phishing_sample:
    if performwhois(url):
        print(counter)
        counter = counter + 1
        phishing_urls.append(url)

0
1
2
Error trying to connect to socket: closing socket
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
Error trying to connect to socket: closing socket
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179


In [ ]:
phishing_df = pd.DataFrame()

In [ ]:
phishing_df["url"] = phishing_urls

In [ ]:
phishing_df["label"] = 1

In [ ]:
phishing_df.to_csv("/content/drive/MyDrive/URL_classification using ML/Training/Data/whois_verified_phishing_training(short).csv",index=False)

# Feature Extraction from training URLs
Here we extract the whois features and some lexical features from every URL in the training dataset to train the model.

In [22]:
!pip install tldextract

     |████████████████████████████████| 93 kB 1.5 MB/s 


In [23]:
import datetime
from os.path import splitext
import tldextract
from urllib.parse import urlparse

In [24]:
#loading dataset which contains 250 phishing URLs and 250 legitimate URLs
dataset = pd.read_csv("/content/drive/MyDrive/URL_classification using ML/Training/Data/merged_whois_verified_training(short).csv")

Extracting whois features from URLs

In [25]:
#function to fetch the website age in days using URL created_date
def get_registered_date_in_days(whois_result):
    if(whois_result!=False):
        created_date = whois_result.creation_date
        if((created_date is not None) and (type(created_date)!=str)):
            if(type(created_date)==list):
                created_date=created_date[0]
            today_date=datetime.datetime.now()
            days = (today_date-created_date).days
            return days
        else:
            return -1
    else:
        return -1

In [26]:
#function to fetch the website expiry date in days using URL expiration_date
def get_expiration_date_in_days(whois_result):
    if(whois_result!=False):
        expiration_date = whois_result.expiration_date
        if((expiration_date is not None) and (type(expiration_date)!=str)):
            if(type(expiration_date)==list):
                expiration_date = expiration_date[0]
            today_date=datetime.datetime.now()
            days = (expiration_date-today_date).days
            return days
        else:
            return -1
    else:
        return -1

In [27]:
#function to fetch the website's last updated date in days using URL updated_date
def get_updated_date_in_days(whois_result):
    if(whois_result!=False):
        updated_date = whois_result.updated_date
        if((updated_date is not None) and (type(updated_date)!=str)):
            if(type(updated_date)==list):
                updated_date = updated_date[0]
            today_date=datetime.datetime.now()
            days = (today_date-updated_date).days
            return days
        else:
            return -1
    else:
        return -1

Extracting Lexical Features from URLs

In [28]:
def get_dot_count(url):
    return url.count('.')

In [29]:
def get_url_length(url):
    return len(url)

In [30]:
def get_digit_count(url):
    return sum(c.isdigit() for c in url)

In [31]:
def get_special_char_count(url):
    count = 0
    special_characters = [';','+=','_','?','=','&','[',']']
    for each_letter in url:
        if each_letter in special_characters:
            count = count + 1
    return count

In [32]:
def get_hyphen_count(url):
    return url.count('-')

In [33]:
def get_double_slash(url):
    return url.count('//')

In [34]:
def get_single_slash(url):
    return url.count('/')

In [35]:
def get_at_the_rate(url):
    return url.count('@')

In [36]:
def get_protocol(url):
    protocol = urlparse(url)
    if(protocol.scheme == 'http'):
        return 1
    else:
        return 0

In [37]:
def get_protocol_count(url):
    http_count = url.count('http')
    https_count = url.count('https')
    http_count = http_count - https_count #correcting the miscount of https as http
    return (http_count + https_count)

In [38]:
registered_date_in_days = []
expiration_date_in_days = []
updated_date_in_days = []
dotCount = []
urlLength = []
digitCount = []
specialCharCount = []
hyphenCount = []
doubleSlashCount = []
singleSlashCount = []
atTheRateCount = []
protocol = []
protocolCount = []

In [39]:
def extract_all_features():
    counter = 0
    for url in dataset['url']:
        counter = counter + 1
        print(counter)
        whois_result = performwhois(url)
        #Extracting whois features from URLs
        registered_date_in_days.append(get_registered_date_in_days(whois_result))
        expiration_date_in_days.append(get_expiration_date_in_days(whois_result))
        updated_date_in_days.append(get_updated_date_in_days(whois_result))
        #Extracting lexical features from URLs
        dotCount.append(get_dot_count(url))
        urlLength.append(get_url_length(url))
        digitCount.append(get_digit_count(url))
        specialCharCount.append(get_special_char_count(url))
        hyphenCount.append(get_hyphen_count(url))
        doubleSlashCount.append(get_double_slash(url))
        singleSlashCount.append(get_single_slash(url))
        atTheRateCount.append(get_at_the_rate(url))
        protocol.append(get_protocol(url))
        protocolCount.append(get_protocol_count(url))

In [40]:
extract_all_features()

1
2
3
4
Error trying to connect to socket: closing socket
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
Error trying to connect to socket: closing socket
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
Error trying to connect to socket: c

In [41]:
print(f'Registered Date list length               : {len(registered_date_in_days)}')
print(f'Expiration Date list length               : {len(expiration_date_in_days)}')
print(f'Updation Date list length                 : {len(updated_date_in_days)}')
print(f'Dot Count list length                     : {len(dotCount)}')
print(f'URL Length list length                    : {len(urlLength)}')
print(f'Digit Count list length                   : {len(digitCount)}')
print(f'Special Character Count list length       : {len(specialCharCount)}')
print(f'Hyphen Count list length                  : {len(hyphenCount)}')
print(f'Double Slash Count list length            : {len(doubleSlashCount)}')
print(f'Single Slash Count list length            : {len(singleSlashCount)}')
print(f'At the Rate(@) Count list length          : {len(atTheRateCount)}')
print(f'ProtocolName Count list length            : {len(protocol)}')
print(f'Protocol Count list length                : {len(protocolCount)}')

Registered Date list length               : 501
Expiration Date list length               : 501
Updation Date list length                 : 501
Dot Count list length                     : 501
URL Length list length                    : 501
Digit Count list length                   : 501
Special Character Count list length       : 501
Hyphen Count list length                  : 501
Double Slash Count list length            : 501
Single Slash Count list length            : 501
At the Rate(@) Count list length          : 501
ProtocolName Count list length            : 501
Protocol Count list length                : 501


In [42]:
features_df = pd.DataFrame()
features_df['whois_regDate'] = registered_date_in_days
features_df['whois_expDate'] = expiration_date_in_days
features_df['whois_updatedDate'] = updated_date_in_days
features_df["dot_count"] = dotCount
features_df["url_len"] = urlLength
features_df["digit_count"] = digitCount
features_df["special_count"] = specialCharCount
features_df["hyphen_count"] = hyphenCount
features_df["double_slash"] = doubleSlashCount
features_df["single_slash"] = singleSlashCount
features_df["at_the_rate"] = atTheRateCount
features_df["protocol"] = protocol
features_df["protocol_count"] = protocolCount

In [43]:
features_df.to_csv("/content/drive/MyDrive/URL_classification using ML/Training/Features/features(short).csv",index=False)